In [2]:
%load_ext autoreload
%autoreload 2

In [21]:
import os
import numpy as np
import parse
from model import *

In [24]:
replay = os.path.join( "train", os.listdir("train")[0])
data = parse.parse_replay_file(replay)

# data = parse.parse_replay_folder("train", "Daniel_Kukiela", max_files=10)

Load Replay: train\ts2018-halite-3-gold-replays_replay-20181009-142856+0000-1539095317-32-32
Parsing data for [{'energy': 5000, 'entities': [], 'factory_location': {'x': 8, 'y': 8}, 'name': 'tsadmiral v1', 'player_id': 0}, {'energy': 5000, 'entities': [], 'factory_location': {'x': 23, 'y': 8}, 'name': 'anglee v2', 'player_id': 1}, {'energy': 5000, 'entities': [], 'factory_location': {'x': 8, 'y': 23}, 'name': 'jon-stumpf v6', 'player_id': 2}, {'energy': 5000, 'entities': [], 'factory_location': {'x': 23, 'y': 23}, 'name': 'Daniel_Kukiela v5', 'player_id': 3}]



  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

Load Basic Information for {'energy': 5000, 'entities': [], 'factory_location': {'x': 8, 'y': 8}, 'name': 'tsadmiral v1', 'player_id': 0}
Load Cell Information
Load Player Ships
Load Other Player Ships
Load Droppoff Information



 25%|█████████████████████                                                               | 1/4 [00:14<00:43, 14.42s/it]

Load Basic Information for {'energy': 5000, 'entities': [], 'factory_location': {'x': 23, 'y': 8}, 'name': 'anglee v2', 'player_id': 1}
Load Cell Information
Load Player Ships
Load Other Player Ships
Load Droppoff Information



 50%|██████████████████████████████████████████                                          | 2/4 [00:29<00:28, 14.47s/it]

Load Basic Information for {'energy': 5000, 'entities': [], 'factory_location': {'x': 8, 'y': 23}, 'name': 'jon-stumpf v6', 'player_id': 2}
Load Cell Information
Load Player Ships
Load Other Player Ships
Load Droppoff Information



 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:41<00:13, 13.81s/it]

Load Basic Information for {'energy': 5000, 'entities': [], 'factory_location': {'x': 23, 'y': 23}, 'name': 'Daniel_Kukiela v5', 'player_id': 3}
Load Cell Information
Load Player Ships
Load Other Player Ships
Load Droppoff Information



100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:55<00:00, 13.86s/it]


In [29]:
game_states = []
for player_data in data:
    game_states.append(HaliteModel.process_game_states(player_data))

Processing Game States
Processing Game States
Processing Game States
Processing Game States


In [30]:
m = HaliteModel(data=data)
features, labels = [], []

for states in game_states:
    f, l = m.generate_training_data(states)
    features.extend(f)
    labels.extend(l)

Generating Training Data



100%|██████████████████████████████████████████████████████████████████████████████| 2931/2931 [03:03<00:00, 16.01it/s]

Generating Training Data



100%|██████████████████████████████████████████████████████████████████████████████| 1223/1223 [01:11<00:00, 17.03it/s]

Generating Training Data



100%|██████████████████████████████████████████████████████████████████████████████| 1106/1106 [01:19<00:00, 10.49it/s]


Generating Training Data



100%|██████████████████████████████████████████████████████████████████████████████| 2557/2557 [02:53<00:00, 26.07it/s]


In [31]:
features = np.array(features)
labels = np.array(labels)

features.shape, labels.shape

((24196, 78), (24196,))

In [32]:
from sklearn.linear_model.perceptron import Perceptron

model = Perceptron()
model.fit(features, labels)

C:\Users\jacob\Anaconda3\envs\labx\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
      max_iter=None, n_iter=None, n_jobs=1, penalty=None, random_state=0,
      shuffle=True, tol=None, verbose=0, warm_start=False)

In [33]:
model.predict(features[0].reshape(1,-1))[0], labels[0]

(3, 3)